In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.python.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import toimage
from IPython.display import Image
%matplotlib inline 

def renderTrainingResults(history):
  if 'val_acc' in history.history:
    print('val_acc', history.history['val_acc'][-1], 'val_loss', history.history['val_loss'][-1])
  print('acc', history.history['acc'][-1], 'loss', history.history['loss'][-1])
  
  plt.plot(history.history['acc'], 
           label='Доля верных ответов на обучающем наборе')
  if 'val_acc' in history.history:
    plt.plot(history.history['val_acc'], 
           label='Доля верных ответов на проверочном наборе')
  plt.xlabel('Эпоха обучения')
  plt.ylabel('Доля верных ответов')
  plt.legend()
  plt.show()

## Загрузка данных с Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!df -h

In [ ]:
!cp /content/gdrive/'My Drive'/socionic_dataset/ethical-logical-photos.zip .
!ls

ethical-logical-photos.zip  gdrive  sample_data


In [ ]:
!mkdir ./ethical-logical-photos
!unzip ethical-logical-photos.zip -d ethical-logical-photos

In [ ]:
!ls ethical-logical-photos/

In [ ]:
Image('ethical-logical-photos/Logical/32.jpg')

In [ ]:
Image('ethical-logical-photos/Ethical/Ivan Ohlobistin_6.jpg')

## Создаем генератор изображений

In [ ]:
train_dir = 'ethical-logical-photos'
img_width, img_height = 500, 500
input_shape = (img_width, img_height, 3)
batch_size = 32

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255,
                             rotation_range=10,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=True,
                             zoom_range=0.1,
                             validation_split=0.2)

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    class_mode='binary',
    subset='training')

In [ ]:
train_generator.class_indices

In [ ]:
val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary',
    subset='validation')

In [ ]:
val_generator.class_indices

In [ ]:
nb_train_samples = 973
nb_val_samples = 242

## Создаем и обучаем составную нейросеть на основе VGG16


In [ ]:
vgg16_net = VGG16(weights='imagenet', 
                  include_top=False, 
                  input_shape=input_shape)
vgg16_net.trainable = False

In [ ]:
сheckpoint = ModelCheckpoint('ethical-logical-photos.h5',
                              monitor='val_acc', 
                              save_best_only=True,
                              verbose=1)

early_stopping = EarlyStopping(monitor='val_acc', 
                               patience=10)

model = Sequential()
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=50,
    validation_data=val_generator,
    validation_steps=nb_val_samples // batch_size,
    verbose=2,
    callbacks=[сheckpoint, early_stopping]
)

renderTrainingResults(history)

In [ ]:
# Загрузка сохраненых весов с наилучшим результатом
files.download('is-animated-vgg16.h5')

In [ ]:
files.upload()

In [ ]:
# Загрузка модели с сохраненными весами:

model = Sequential()
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.load_weights("is-animated-vgg16.h5")

model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['categorical_accuracy'])

### Тонкая настройка сети (для VGG16) 

"Размораживаем" последний сверточный блок сети VGG16

In [ ]:
vgg16_net.trainable = True
trainable = False
for layer in vgg16_net.layers:
    if layer.name == 'block4_conv1':
        trainable = True
    layer.trainable = trainable

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-6), 
              metrics=['accuracy'])

In [ ]:
# Проверяем количество обучаемых параметров
model.summary()

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=4,
    #validation_data=val_generator,
)

## Создаем и обучаем составную нейронную сеть на основе Inception

In [ ]:
inception = InceptionV3(weights='imagenet', include_top=False)
for layer in inception.layers:
    layer.trainable = False

In [ ]:
сheckpoint = ModelCheckpoint('ethical-logical-photos.h5',
                              monitor='val_acc', 
                              save_best_only=True,
                              verbose=1)

x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=inception.input, outputs=predictions)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=20,
    validation_data=val_generator,
    verbose=1,
    callbacks=[сheckpoint]
)

renderTrainingResults(history)

## Проверка качества работы обученной сети на нескольких образцах

In [ ]:
files.upload()
!unzip test_samples.zip
!ls

In [ ]:
import glob

test_images_paths = glob.glob('test_samples/' + "*.jpg")

for image_path in test_images_paths:
  img = image.load_img(image_path)
  img = img.resize((img_width, img_height))

  predictions = model.predict(np.expand_dims(img, axis=0))
  prediction = predictions[0][0]
  prediction = prediction > 0.5
  
  print('----------')
  print('Тип личности по фото распознано как', '"Logical":' if prediction else '"Ethical":')
  plt.imshow(img)
  plt.show()